# Baseline: Popularity-based Recommender

Цель ноутбука:
- построить простую рекомендательную модель на основе популярности товаров;
- оценить её качество по метрикам `precision@k` и `recall@k` на тестовой выборке;
- зафиксировать базовый уровень качества для сравнения с более сложными моделями.


In [1]:
import pandas as pd
import numpy as np

train = pd.read_csv("../data/processed/train.csv")
val = pd.read_csv("../data/processed/val.csv")
test = pd.read_csv("../data/processed/test.csv")

for df in (train, val, test):
    df["timestamp"] = pd.to_datetime(df["timestamp"])

len(train), len(val), len(test), train["event"].value_counts()


(1008982,
 126123,
 126123,
 event
 view           945976
 addtocart       46036
 transaction     16970
 Name: count, dtype: int64)

In [2]:
train_trans = train[train["event"] == "transaction"]
test_trans = test[test["event"] == "transaction"]

len(train_trans), len(test_trans)


(16970, 2254)

In [3]:
item_popularity = (
    train_trans["itemid"]
    .value_counts()
)

item_popularity.head(10)


itemid
461686    88
119736    74
213834    57
445351    41
48030     39
7943      38
420960    35
17478     32
369447    28
416017    28
Name: count, dtype: int64

In [4]:
TOP_N = 100
top_items = item_popularity.index[:TOP_N].tolist()
top_items[:10]


[461686, 119736, 213834, 445351, 48030, 7943, 420960, 17478, 369447, 416017]

In [5]:
test_user2items = (
    test_trans
    .groupby("visitorid")["itemid"]
    .apply(set)
)

len(test_user2items)


1198

In [6]:
from typing import Iterable, Set, Tuple


def precision_recall_at_k(
    recommended: Iterable[int],
    relevant: Set[int],
    k: int,
) -> Tuple[float, float]:
    """
    recommended: упорядоченный список рекомендованных item_id
    relevant: множество "правильных" item_id (покупок пользователя)
    k: размер топа, по которому считаем метрики
    """
    if k == 0:
        return 0.0, 0.0

    rec_k = list(recommended)[:k]
    rec_set = set(rec_k)

    hit_count = len(rec_set & relevant)
    precision = hit_count / k
    recall = hit_count / len(relevant) if relevant else 0.0

    return precision, recall


In [7]:
def evaluate_popularity_baseline(
    user2items: pd.Series,
    popular_items: list[int],
    ks=(5, 10, 20),
) -> pd.DataFrame:
    """
    user2items: Series[visitorid -> set(itemid)]
    popular_items: глобальный топ товаров по популярности
    ks: набор k для метрик
    """
    rows = []

    for k in ks:
        precisions = []
        recalls = []

        for _, relevant_items in user2items.items():
            p, r = precision_recall_at_k(popular_items, relevant_items, k)
            precisions.append(p)
            recalls.append(r)

        rows.append({
            "k": k,
            "precision@k": np.mean(precisions),
            "recall@k": np.mean(recalls),
        })

    return pd.DataFrame(rows)


results = evaluate_popularity_baseline(
    user2items=test_user2items,
    popular_items=top_items,
    ks=(5, 10, 20),
)

results


,k,precision@k,recall@k
0,5,0.009182,0.021804
1,10,0.005259,0.026569
2,20,0.003297,0.034098


In [8]:
all_items = train["itemid"].unique().tolist()

def evaluate_random_baseline(
    user2items: pd.Series,
    all_items: list[int],
    ks=(5, 10, 20),
    seed: int = 42,
) -> pd.DataFrame:
    rng = np.random.default_rng(seed)
    rows = []

    for k in ks:
        precisions = []
        recalls = []

        for _, relevant_items in user2items.items():
            rec_items = rng.choice(all_items, size=k, replace=False)
            p, r = precision_recall_at_k(rec_items, relevant_items, k)
            precisions.append(p)
            recalls.append(r)

        rows.append({
            "k": k,
            "precision@k": np.mean(precisions),
            "recall@k": np.mean(recalls),
        })

    return pd.DataFrame(rows)


random_results = evaluate_random_baseline(
    user2items=test_user2items,
    all_items=all_items,
    ks=(5, 10, 20),
)

random_results


,k,precision@k,recall@k
0,5,0.000000,0.000000
1,10,0.000000,0.000000
2,20,0.000042,0.000835


In [9]:
baseline_compare = results.copy()
baseline_compare["precision@k (random)"] = random_results["precision@k"]
baseline_compare["recall@k (random)"] = random_results["recall@k"]
baseline_compare


,k,precision@k,recall@k,precision@k (random),recall@k (random)
0,5,0.009182,0.021804,0.000000,0.000000
1,10,0.005259,0.026569,0.000000,0.000000
2,20,0.003297,0.034098,0.000042,0.000835


## Результаты baseline-модели (популярность)

Для baseline использована простейшая стратегия рекомендаций:
всем пользователям предлагается один и тот же список из `TOP_N = 100`
наиболее популярных товаров по числу покупок в train-выборке.

Оценка проводилась на пользователях, у которых есть хотя бы одна покупка в test.

### Метрики качества

| k  | precision@k | recall@k |
|----|-------------|----------|
| 5  | 0.0092      | 0.0218   |
| 10 | 0.0053      | 0.0266   |
| 20 | 0.0033      | 0.0341   |

Для сравнения был рассчитан случайный baseline, который показал
практически нулевые значения precision@k и recall@k.

### Интерпретация

- Несмотря на низкие абсолютные значения метрик, популярностная модель
  значительно превосходит случайные рекомендации.
- Низкие значения precision@k ожидаемы для реального e-commerce датасета
  с большим каталогом товаров и редкими покупками.
- Полученные результаты задают базовый уровень качества,
  который должны превзойти более сложные модели рекомендаций.

**Вывод:**  
Baseline-модель по популярности является корректной отправной точкой
для дальнейшего сравнения с collaborative filtering и обучаемыми моделями.
